<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## 1. Download & Install required libraries

In [3]:
!conda install -c anaconda beautifulsoup4 --yes # Install BeautifulSoup4 (for scraping web content)

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h14c3975_0         3.5 MB  anaconda
    certifi-2018.8.24          |           py36_1         140 KB  anaconda
    ca-certificates-2018.03.07 |                0         124 KB  anaconda
    conda-4.5.11               |           py36_0         1.0 MB  anaconda
    beautifulsoup4-4.6.3       |           py36_0         138 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.9 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py36h49b8c8c_1             --> 4.6.3-py36_0      anaconda
    conda:           4.5.11-py36_0        conda-forge --> 4.5.11-py36_0     anac

In [4]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print ('Geopy installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.1

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print ('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         592 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1001 conda-forge
    branca:  0.3.0-py_0      conda-forge
    folium:  0.5.0-py_0      conda-forge
    vincent: 0.4.4-py_1      conda-forge


altair-2.2.2         | 494 KB    | #######################

In [7]:
!conda install -c anaconda lxml --yes #The lxml XML toolkit is a Pythonic binding for the C libraries libxml2 and libxslt.
print ('LXML installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.2.5                 |   py36hefd8a0e_0         1.6 MB  anaconda
    libgcc-ng-8.2.0            |       hdf63c60_1         7.6 MB  anaconda
    ------------------------------------------------------------
                                           Total:         9.2 MB

The following packages will be UPDATED:

    libgcc-ng:       7.2.0-hdf63c60_3                  --> 8.2.0-hdf63c60_1     anaconda
    lxml:            4.2.1-py36h23eabaa_0              --> 4.2.5-py36hefd8a0e_0 anaconda

The following packages will be DOWNGRADED:

    ca-certificates: 2018.10.15-ha4d7672_0 conda-forge --> 2018.03.07-0         anaconda
    certifi:         2018.10.15-py36_1000  conda-forge --> 2018.8.24-py36_1     anaconda

In [9]:
!conda install -c anaconda html5lib --yes #html5lib is a pure-python library for parsing HTML.
print('HTML5Lib installed')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    html5lib-0.9999999         |           py36_0         176 KB  anaconda

The following packages will be UPDATED:

    html5lib: 0.9999999-py36_0 conda-forge --> 0.9999999-py36_0 anaconda


html5lib-0.9999999   | 176 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
HTML5Lib installed


### Import required libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## 2. Download HTML content to a dataframe

In [40]:
from bs4 import BeautifulSoup

html_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_source, 'lxml') #convert to Soup format

In [44]:
#Get data from the Toronto table
toronto_table = soup.find('table', class_='wikitable sortable')

In [54]:
#Define lists to store required information
postalcodes=[]
boroughs=[]
neighborhoods=[]

# Fill data from toronto_table to dataframe
for row in toronto_table.find_all('tr'):
    cells = row.find_all('td')
    if (len(cells) != 0): #to skip empty content
        
        postalcode = cells[0].text.strip() #strip() function to remove '\n' character
        borough = cells[1].text.strip()
        neighborhood = cells[2].text.strip()
        
        if(borough != 'Not assigned'): # to skip row which has BOROUGH = 'Not assigned'
            #get individual value
            postalcodes.append(postalcode) 
            boroughs.append(borough)
            neighborhoods.append(neighborhood)

#Assign values to a dataframe
toronto_df = pd.DataFrame({'PostalCode':postalcodes, 'Borough':boroughs, 'Neighborhood':neighborhoods})

toronto_df.head(20)
        

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## 3. Manipulate the dataframe

In [69]:
# First, get all unique Postal Code
u_postalcodes = toronto_df['PostalCode'].unique()
u_postalcodes = u_postalcodes.tolist() # convert to Python list from Numpy array

In [74]:
# Secondly, get all Borough(s) corresponded to those PostalCodes above
u_boroughs = []
for pc in u_postalcodes:
    u_boroughs.append(toronto_df[toronto_df.PostalCode == pc]['Borough'].unique().tolist()[0])

<class 'list'>


In [86]:
# Thirdly, concatenate all Neighborhoods related to the PostalCodes 
u_neighborhoods = []
for pc in u_postalcodes:
    all_neighborhoods = toronto_df[toronto_df.PostalCode == pc]['Neighborhood'].tolist()
    temp = ''
    for nbh in all_neighborhoods:
        temp += nbh + ', '
    
    u_neighborhoods.append(temp[0:len(temp)-2]) # -2 to remove the ' ,' term
    
#u_neighborhoods

In [89]:
# Reconstruct our dataframe
toronto_df = pd.DataFrame({'PostalCode':u_postalcodes, 'Borough':u_boroughs, 'Neighborhood':u_neighborhoods})
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [96]:
# Get all neiborhood which has value = 'Not assigned'
toronto_df[toronto_df.Neighborhood == 'Not assigned']

# Since there's only 1 record which has Neighborhood = 'Not assigned', we can fix our dataframe easily:
toronto_df.at[4, 'Neighborhood'] = "Queen's Park"

## 4. Finally, here's our dataframe

In [97]:
# Finally, here's our dataframe
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Shape of our dataframe

In [99]:
# Shape of our dataframe
toronto_df.shape

(103, 3)

## 5. Add Latitude & Longitude

In [106]:
# Load latitude & longitude from CSV

geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
print('Loaded')

Loaded


In [117]:
# Rename columns of geo_df
geo_df.columns = ['PostalCode', 'Latitude', 'Longitude']

# Get latitude & longitude corresponded to our PostalCode
latitudes = []
longitudes = []

for u in u_postalcodes:
    latitudes.append(geo_df[geo_df.PostalCode == u]['Latitude'].tolist()[0])
    longitudes.append(geo_df[geo_df.PostalCode == u]['Longitude'].tolist()[0])

### Our new Dataframe

In [121]:
updated_toronto_df = toronto_df.copy()
updated_toronto_df['Latitude'] = latitudes
updated_toronto_df['Longitude'] = longitudes

updated_toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
